## Test uniswap 1&2
Prueba de obtención de datos de UNISWAP 1 y 2 para un token.

- Obtención del exchange de ese token
- Obtención de precio para una cantidad de ese token
- Comparación de precios (arbitrage)

Para la prueba necesitamos una cuenta en [Infura.io](https://infura.io) para no tener que desplegar un nodo local de Ethereum.

### Imports

Para interactuar con el blockchain necesitamos:
- Módulo web3 (pip install web3)
- ipython_secrets para no llenar el código de información sensible.


In [1]:
from web3 import Web3
from ipython_secrets import *

### Contratos y ABI

Para interactuar con el blockchain necesitamos:
- Dirección del contrato
- ABI (interfaz) de cada contrato.

En nuestro caso necesitamos 3 ABIs distintas:

- Uniswap1 *exchange* ABI: Aquí es donde se compra y vende el token.
- Uniswap2 *exchange* ABI: Ídem para Uniswap2.
- ERC20 ABI: Para averiguar el balance de cada exchange.


Todo esto está en [definitions.py](./definitions.py)


In [2]:
from definitions import *

### Cliente W3

Para interactuar con el blockchain tenemos que intanciar un cliente.

Usamos infura con transporte de Websocket.

In [3]:

infura_uri = f'wss://mainnet.infura.io/ws/v3/{get_secret("INFURA_API_KEY")}'  # SECRET
w3 = Web3(Web3.WebsocketProvider(infura_uri))

## Interactuamos con los contratos

In [4]:
token_addr = '0x3A9FfF453d50D4Ac52A6890647b823379ba36B9E'
weth_addr = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
uniswap1_addr = '0x536956Fab86774fb55CfaAcF496BC25E4d2B435C'
uniswap2_addr = '0x260E069deAd76baAC587B5141bB606Ef8b9Bab6c'

In [5]:

token_addr_cs =  Web3.toChecksumAddress(token_addr)  # Checksum address por motivos de seguridad.
token_contract = w3.eth.contract(address=token_addr_cs, abi=erc20_ABI)
weth_contract = w3.eth.contract(address=weth_addr, abi=erc20_ABI)


In [6]:
### Informacion del token

token_name = token_contract.functions.name().call()
token_symbol = token_contract.functions.symbol().call()
token_decimals = token_contract.functions.decimals().call() 

print(f'ERC-20 Token name: { token_name } ({token_symbol})')
print(f'Decimals: { token_decimals } ( { 10 ** -token_decimals} )')


ERC-20 Token name: Shuffle.Monster V3 (SHUF)
Decimals: 18 ( 1e-18 )


In [7]:
# Balance de los contratos UNISWAP

print('UniswapV1 balances:')

token_balancev1 = token_contract.functions.balanceOf(uniswap1_addr).call() * ( 10 ** -token_decimals )
eth_balancev1 = w3.eth.getBalance(uniswap1_addr) * ( 1e-18 )
print(f'   { token_balancev1} { token_symbol} ')
print(f'   { eth_balancev1} ETH ')
print(f'\nUniswap1 Price:  { eth_balancev1 / ( token_balancev1 ) } { token_symbol } / ETH')


print('\nUniswapV2 balances:')

token_balancev2 = token_contract.functions.balanceOf(uniswap2_addr).call() * ( 10 ** -token_decimals )
eth_balancev2 = w3.eth.getBalance(uniswap2_addr) * ( 1e-18 )
weth_balancev2 = weth_contract.functions.balanceOf(uniswap2_addr).call() * ( 1e-18 )

print(f'   { token_balancev2} { token_symbol} ')
print(f'   { eth_balancev2} ETH ')
print(f'   { weth_balancev2} WETH')
print(f'\n Uniswap2 Price: { weth_balancev2 / ( token_balancev2 ) } { token_symbol } / ETH')



UniswapV1 balances:
   4579.065019173647 SHUF 
   2.1950382410938216 ETH 

Uniswap1 Price:  0.00047936385089590746 SHUF / ETH

UniswapV2 balances:
   89791.56886166017 SHUF 
   0.0 ETH 
   37.4352879509202 WETH

 Uniswap2 Price: 0.000416913173759063 SHUF / ETH
